In [4]:
# Import relevant packages
import numpy as np
import os
from shutil import copyfile
from zipfile import ZipFile

# Download NIH dataset zip file
!wget -nc ftp://lhcftp.nlm.nih.gov/Open-Access-Datasets/Malaria/cell_images.zip

# Extract images if not already extracted
ROOT_DIR = os.path.join("/", "content")
if not os.path.isdir("cell_images"):
    print("Extracting images...")
    with ZipFile(os.path.join("cell_images.zip"), "r") as zipObj:
        zipObj.extractall()
    print("Done!")

# Install and import relevant packages
import numpy as np
import os
!pip install opencv-python
!apt update && apt install -y libsm6 libxext6 libxrender1
import cv2
from PIL import Image

# Create new folders to save rescaled images
if not os.path.isdir("RescaledSet"):
    os.mkdir("RescaledSet")
if not os.path.isdir("RescaledSet/Parasitized"):
    os.mkdir("RescaledSet/Parasitized")
if not os.path.isdir("RescaledSet/Uninfected"):
    os.mkdir("RescaledSet/Uninfected")

# Generate list of parasitized file names
ParasitizedFiles = os.listdir("cell_images/Parasitized/")
UninfectedFiles = os.listdir("cell_images/Uninfected/")

# Remove Thumb.db files
while 'Thumbs.db' in ParasitizedFiles: ParasitizedFiles.remove('Thumbs.db')   
while 'Thumbs.db' in UninfectedFiles: UninfectedFiles.remove('Thumbs.db')  

# Pre-allocate memory space for images
Parasitized = np.empty([13779,128,128,3])
Uninfected = np.empty([13779,128,128,3])

# Resize and load parasitized images
for i in range(13779):
    TempImage = cv2.imread('cell_images/Parasitized/'+ParasitizedFiles[i])
    ResizedImage = cv2.resize(TempImage, dsize=(128,128))
    Parasitized[i,:,:,:] = ResizedImage

# Resize and load uninfected images
for i in range(13779):
    TempImage = cv2.imread('cell_images/Uninfected/'+UninfectedFiles[i])
    ResizedImage = cv2.resize(TempImage, dsize=(128,128))
    Uninfected[i,:,:,:] = ResizedImage
    
print('Uninfected Dataset size is:',np.shape(Uninfected))
print('Parasitized Dataset size is:',np.shape(Parasitized))

# Generate dataset labels
ParasitizedLabels = np.repeat([[0,1]], 13779, axis=0)
UninfectedLabels = np.repeat([[1,0]], 13779, axis=0)
Labels = np.concatenate((ParasitizedLabels,UninfectedLabels), axis=0)

# Generate image dataset
Dataset = np.concatenate((Parasitized, Uninfected), axis=0)

# Generate 5-fold cross-validation groups
CVIndices = np.random.permutation(Dataset.shape[0])
TrainInd, TestInd = CVIndices[:22046], CVIndices[22046:]

# Generate train and test sets
from skimage.transform import rescale, resize, downscale_local_mean

TrainOut = Dataset[TrainInd,:]
TestOut = Dataset[TestInd,:]
TrainIn = np.zeros([np.shape(TrainOut)[0],32,32,3])
TestIn = np.zeros([np.shape(TestOut)[0],32,32,3])
for i in range(np.shape(TrainOut)[0]):
    TrainIn[i,:,:,:] = downscale_local_mean(TrainOut[i,:,:,:], (4,4,1))
for i in range(np.shape(TestOut)[0]):
    TestIn[i,:,:,:] = downscale_local_mean(TestOut[i,:,:,:], (4,4,1))

# del Dataset
del Parasitized
del Uninfected

File ‘cell_images.zip’ already there; not retrieving.
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease             
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease         
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done33m             
Building dependency tree       
Reading state information... Done
28 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsm6

In [12]:
# Generate train and test sets
! pip install scikit-image
from skimage.transform import rescale, resize, downscale_local_mean

del TrainOut
del TrainIn
del TestOut
del TestIn
del model

CVIndices = np.random.permutation(Dataset.shape[0])
TestInd, TrainInd = CVIndices[22046:], CVIndices[:22046]
TrainOut = Dataset[TrainInd,:]
TestOut = Dataset[TestInd,:]
TrainIn = np.zeros([np.shape(TrainOut)[0],32,32,3])
TestIn = np.zeros([np.shape(TestOut)[0],32,32,3])
for i in range(np.shape(TrainOut)[0]):
    TrainIn[i,:,:,:] = downscale_local_mean(TrainOut[i,:,:,:], (4,4,1))
for i in range(np.shape(TestOut)[0]):
    TestIn[i,:,:,:] = downscale_local_mean(TestOut[i,:,:,:], (4,4,1))


In [ ]:
## Create FSRCNN
from keras import optimizers
from keras.models import load_model
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Input, ZeroPadding2D, Conv2DTranspose, merge # Merge
from keras.layers.advanced_activations import PReLU
from keras.preprocessing import image

model = Sequential()

input_img = Input(shape=(32,32,3))
model = Conv2D(56, (5, 5), padding='same', kernel_initializer='he_normal')(input_img)
model = PReLU()(model)
model = Conv2D(16, (1, 1), padding='same', kernel_initializer='he_normal')(model)
model = PReLU()(model)
model = Conv2D(12, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = PReLU()(model)
model = Conv2D(12, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = PReLU()(model)
model = Conv2D(12, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = PReLU()(model)
model = Conv2D(12, (3, 3), padding='same', kernel_initializer='he_normal')(model)
model = PReLU()(model)
model = Conv2D(56, (1, 1), padding='same', kernel_initializer='he_normal')(model)
model = PReLU()(model)
model = Conv2DTranspose(3, (9, 9), strides=(4, 4), padding='same')(model)
output_img = model

model = Model(input_img, output_img)
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss = "mean_squared_error", optimizer = adam, metrics=["mean_squared_error"])

model.summary()

Results = model.fit(TrainIn, TrainOut, epochs=100, batch_size=32, validation_data=(TestIn,TestOut), validation_freq=1)

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 32, 32, 56)        4256      
_________________________________________________________________
p_re_lu_22 (PReLU)           (None, 32, 32, 56)        57344     
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 32, 32, 16)        912       
_________________________________________________________________
p_re_lu_23 (PReLU)           (None, 32, 32, 16)        16384     
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 32, 32, 12)        1740      
_________________________________________________________________
p_re_lu_24 (PReLU)           (None, 32, 32, 12)        1228

In [11]:
# Display and store performance results
Results.history['loss'] = [round(l, 4) for l in Results.history['loss']]
Results.history['val_loss'] = [round(l, 4) for l in Results.history['val_loss']]

print('Training Loss:',Results.history['loss'])
print('Validation Loss:',Results.history['val_loss'])

Training Loss: [603.4778, 177.1807, 140.7971, 127.0634, 118.3863, 112.1636, 107.1357, 104.0426, 100.6233, 97.7789, 95.8339, 93.9094, 92.0101, 90.4347, 88.4681, 87.619, 85.4094, 84.3058, 83.3763, 81.961, 80.854, 80.0754, 78.8659, 78.2094, 77.3011, 76.5185, 75.7019, 75.2506, 74.0739, 73.6494, 73.1564, 72.7383, 71.7818, 71.5512, 71.0196, 70.3811, 69.822, 69.5974, 69.1962, 68.9069, 68.5386, 68.0346, 68.0191, 67.3101, 67.5725, 66.8465, 66.8657, 66.6049, 66.0988, 65.9479, 66.2408, 65.4877, 65.5312, 65.0516, 65.207, 64.7479, 64.7483, 64.6209, 64.1678, 64.15, 64.0719, 63.7552, 63.7174, 63.4913, 63.748, 62.9288, 63.1247, 63.0251, 62.7186, 62.6796, 62.9762, 62.4469, 62.0599, 62.1274, 62.1146, 61.8613, 61.7741, 61.8657, 61.5643, 61.5268, 61.3072, 61.2596, 61.2738, 61.1602, 60.94, 60.9058, 60.5973, 60.8762, 60.5178, 60.555, 60.2824, 60.2501, 60.4073, 59.8514, 60.0124, 60.0175, 59.6977, 59.7476, 59.7148, 59.5472]
Validation Loss: [209.2959, 151.5091, 133.5549, 123.1183, 115.5795, 109.6952, 106.1166

In [ ]:
import sys
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules], key=lambda x: x[1], reverse=True)